<a href="https://colab.research.google.com/github/yjjangg/yjjang/blob/main/Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_03

In [ ]:
import pandas as pd
long_data=pd.read_csv('data/long_data.csv')
long_data

In [ ]:
long_df = pd.read_csv(
    'data/long_data.csv', usecols=['date', 'datatype', 'value']
).rename(
    columns={'value': 'temp_C'}
).assign(
    date=lambda x: pd.to_datetime(x.date),
    temp_F=lambda x: (x.temp_C * 9/5) + 32
)
long_df.head()

In [ ]:
pivot_df=long_df.pivot(
    index='date', columns='datatype', values=['temp_C','temp_F']
)
pivot_df.head()

In [ ]:
pivot_df.describe()

In [ ]:
multi_index_df = long_df.set_index(['date', 'datatype'])
multi_index_df.head().index

In [ ]:
unstacked_df = multi_index_df.unstack()
unstacked_df.head()

In [ ]:
extra_data = long_df.append([{
    'datatype': 'TAVG', 
    'date': '2018-10-01', 
    'temp_C': 10, 
    'temp_F': 50
}]).set_index(['date', 'datatype']).sort_index()

extra_data['2018-10-01':'2018-10-02']

In [ ]:
extra_data.unstack().head()

In [ ]:
wide_df = pd.read_csv('data/wide_data.csv')
wide_df.head()

In [ ]:
melted_df = wide_df.melt(
    id_vars='date',
    value_vars=['TMAX', 'TMIN', 'TOBS'],
    value_name='temp_C',
    var_name='measurement'
)
melted_df.head()

In [ ]:
wide_df.set_index('date', inplace=True)
wide_df.head()

In [ ]:
stacked_series = wide_df.stack()
stacked_series.head()

In [ ]:
stacked_df = stacked_series.to_frame('values')
stacked_df.head()

In [ ]:
df = pd.read_csv('data/dirty_data.csv')
df.head()

In [ ]:
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls

In [ ]:
df[df.inclement_weather.isna()].shape[0]

In [ ]:
import numpy as np
df[df.SNWD.isin([-np.inf, np.inf])]

In [ ]:
df[df.duplicated()]

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
# 1. make the date a datetime
df.date = pd.to_datetime(df.date)

# 2. save this information for later
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. sort ? to the bottom
df.sort_values('station', ascending=False, inplace=True)

# 4. drop duplicates based on the date column keeping the first occurrence 
# which will be the valid station if it has data
df_deduped = df.drop_duplicates('date')

# 5. remove the station column because we are done with it
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. take valid station's WESF and fall back on station ? if it is null
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)


In [ ]:
df_deduped

In [ ]:
df_deduped.dropna().shape

In [ ]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

In [ ]:
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

In [ ]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

df_deduped

In [ ]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill'),
    TOBS=lambda x: x.TOBS.fillna(method='ffill')
).head()

In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

In [ ]:
weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

In [ ]:
prcp_data = weather.query('datatype == "PRCP" and station.str.contains("S0007")',engine='python')
prcp_data.head()

In [ ]:
weather[
    (weather.datatype == 'PRCP')
    & (weather.station.str.contains('S0007'))
]

In [ ]:
prcp_data['attributes'].describe()

In [ ]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join

In [ ]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')


In [ ]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')], 
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0) 
    for kind in outer_join._merge.unique()
]).sort_index()